# word2vec 단어 간 유사도 체크
- 최대한 육아를 주제로 한 말뭉치를 모아보았다. (Total 2,755 documents)
- 우리가 원하는 의도에 일치하는 단어 간 유사도 결과가 나오는지를 확인하는 것이 목적이다.

## Import

In [1]:
import pandas as pd
import numpy as np
import re

from konlpy.tag import Mecab
from konlpy.tag import Komoran
from gensim.models import Word2Vec

### Datas

In [2]:
df = pd.read_csv("data/text_for_word2vec.csv")
df.tail()

,text
24724,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...
24725,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...
24726,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...
24727,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...
24728,최근 전자파가 건강상의 문제를 일으킬 수 있다는 견해가 많아지면서 임신부들 또한 가...


### Add tokenized column

191210 정체불명의 error 발생
```
NotImplementedError: Wrong number or type of arguments for overloaded function 'Tagger_parse'.
  Possible C/C++ prototypes are:
    MeCab::Tagger::parse(MeCab::Model const &,MeCab::Lattice *)
    MeCab::Tagger::parse(MeCab::Lattice *) const
    MeCab::Tagger::parse(char const *)
```

babynews 데이터가 추가된 후 갑자기 발생한 에러인데.. 왜인지 모르겠지만 해당 부분 데이터프레임만 따로 tokenize하는 것은 문제없이 돌아간다.  
따라서 번거롭지만 분리하여 tokenizing 한 후에 합치기로 하였다.  

-> 해결.. `NaN` value 문제였다(어이없게도 NaN 값이 중간에 섞여있었음;). 누락 데이터를 잘 체크하는 습관을 갖도록하자.

In [3]:
def tokenize(text):
    tokenizer = Mecab()
    text = tokenizer.morphs(text)
#     text = ' '.join(text)
    return text

In [4]:
%%time
df['tokenized'] = df['text'].apply(tokenize)
df.tail()

CPU times: user 1min 5s, sys: 30 s, total: 1min 35s
Wall time: 1min 35s


,text,tokenized
24724,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...,"[가, 임기, 연력, 여성, 의, 흡연, 률, 증가, 와, 그, 폐해, 현대, 여성..."
24725,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...,"[아토피, 에, 유전, 적, 인, 요인, 이, 작용, 한다는, 사실, 알, 고, 계..."
24726,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...,"[1, st, 시기, 에, 따라, 필요, 한, 영양소, 를, 충분히, 뱃속, 의, ..."
24727,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...,"[분만, 여성, 만, 이, 누릴, 수, 있, 는, 특권, 대부분, 의, 여성, 들,..."
24728,최근 전자파가 건강상의 문제를 일으킬 수 있다는 견해가 많아지면서 임신부들 또한 가...,"[최근, 전자파, 가, 건강, 상, 의, 문제, 를, 일으킬, 수, 있, 다는, 견..."


In [5]:
df.isnull().sum()

text         0
tokenized    0
dtype: int64

## Train Word2Vec

#### WTF error
- df를 csv 형식으로 저장한 후 다시 로드하면 Word2Vec 학습에 인코딩 문제가 생긴다.
- 더 자세히 파고들어보니 list 형식의 column이 csv로 저장할 때 강제로 string 타입으로 형변환 됨.
- 이렇게 형변환 된 column을 Word2Vec 학습에 사용할 시 한글 텍스트가 깨지는 현상이 발생함.. 정확한 원인은 아직 모르겠다.

In [6]:
# df = pd.read_csv('text_for_word2vec.csv', encoding='utf-8')

그러므로 저장 & 로드 과정 생략하고 바로 말뭉치를 numpy ndarray형식으로 변환

In [7]:
corpus = np.array(df.tokenized)
type(corpus)

numpy.ndarray

In [8]:
%%time
model_all = Word2Vec(corpus,
                     size=200,
                     window=6,
                     min_count=5,
                     workers=4,
                     seed=1,
                     sg=1)
model_all.save('data/tokmom_word2vec_all.bin')

CPU times: user 8min 58s, sys: 966 ms, total: 8min 59s
Wall time: 2min 21s


#### 유사어 분석

In [9]:
model_all.wv.similar_by_word("놀이")

[('소꿉', 0.6915862560272217),
 ('기차놀이', 0.6761323809623718),
 ('공놀이', 0.6412385106086731),
 ('집짓기', 0.6292974948883057),
 ('불록', 0.6187699437141418),
 ('놀잇감', 0.6158180236816406),
 ('음률', 0.615355372428894),
 ('무너뜨리기', 0.6150493621826172),
 ('타악기', 0.6128491163253784),
 ('손유희', 0.6123851537704468)]

In [10]:
model_all.wv.similar_by_word("아이")

[('아기', 0.6805053353309631),
 ('어린아이', 0.6047922372817993),
 ('외둥이', 0.5873357057571411),
 ('엄마', 0.5850118398666382),
 ('들떠서', 0.57811439037323),
 ('안쓰러워서', 0.5776230096817017),
 ('놀림거리', 0.5704067945480347),
 ('달려드', 0.5681945085525513),
 ('길러서', 0.5675917863845825),
 ('갓난아이', 0.5661477446556091)]

In [11]:
model_all.wv.similar_by_word("치아")

[('잇몸', 0.804706335067749),
 ('충치', 0.752939760684967),
 ('영구치', 0.7441245913505554),
 ('젖니', 0.6988855600357056),
 ('턱뼈', 0.6692328453063965),
 ('앞니', 0.6573141813278198),
 ('어금니', 0.6486890912055969),
 ('에나멜질', 0.6472829580307007),
 ('우식증', 0.6406774520874023),
 ('유치', 0.6387851238250732)]

In [12]:
model_all.wv.similar_by_word("이빨")

[('혓바닥', 0.7247183918952942),
 ('거스러미', 0.7225725054740906),
 ('허옇', 0.704049289226532),
 ('뜯어내', 0.7025974988937378),
 ('밤색', 0.6978909969329834),
 ('영인', 0.6970874667167664),
 ('핥', 0.6959443092346191),
 ('고무젖꼭지', 0.6953127980232239),
 ('잽싸', 0.6928242444992065),
 ('갈라진다', 0.6921412944793701)]

In [13]:
model_all.wv.similar_by_word("감기")

[('목감기', 0.792517900466919),
 ('코감기', 0.771400511264801),
 ('기침감기', 0.7424240112304688),
 ('냉방병', 0.7109965085983276),
 ('편도염', 0.7061010003089905),
 ('기관지염', 0.7000141143798828),
 ('인후염', 0.6962705850601196),
 ('콜록콜록', 0.6947083473205566),
 ('독감', 0.6907258629798889),
 ('호흡기', 0.6902254819869995)]

In [14]:
model_all.wv.similar_by_word("열")

[('들어온다고', 0.5407794713973999),
 ('고열로', 0.5392342805862427),
 ('오한', 0.5383657217025757),
 ('심열', 0.5267727375030518),
 ('열기', 0.5254351496696472),
 ('펄펄', 0.521113395690918),
 ('미열', 0.5192996263504028),
 ('고열', 0.5159453749656677),
 ('성경', 0.5083253383636475),
 ('사병', 0.5009409785270691)]

In [15]:
model_all.wv.similar_by_word("아파")

[('아프', 0.761528491973877),
 ('아파서', 0.7283117771148682),
 ('보챈다', 0.68876713514328),
 ('다쳐', 0.6809108257293701),
 ('마려워', 0.6733943223953247),
 ('토해서', 0.6720496416091919),
 ('아파하', 0.6706192493438721),
 ('콜록거리', 0.6675966382026672),
 ('지끈지끈', 0.6671324968338013),
 ('먹먹해', 0.6666386127471924)]

In [16]:
model_all.wv.similar_by_word("아프")

[('아파', 0.761528491973877),
 ('결리', 0.6961172819137573),
 ('아파하', 0.678308367729187),
 ('아픈', 0.6770675778388977),
 ('먹먹해', 0.6763649582862854),
 ('욱신거리', 0.6705669164657593),
 ('쑤시', 0.669019341468811),
 ('시큰거리', 0.6664289236068726),
 ('지끈지끈', 0.6648440361022949),
 ('시큰', 0.6628653407096863)]

In [17]:
model_all.wv.similar_by_word("울음")

[('터뜨릴', 0.6978093385696411),
 ('터뜨린다', 0.6683531403541565),
 ('터뜨립니다', 0.661981463432312),
 ('터뜨린', 0.6549371480941772),
 ('자지러질', 0.6433998346328735),
 ('운다', 0.6268394589424133),
 ('흐느끼', 0.6243926882743835),
 ('터뜨려', 0.624337375164032),
 ('고래고래', 0.6207104921340942),
 ('지르기', 0.6141247749328613)]

In [18]:
model_all.wv.similar_by_word("남아")

[('여아', 0.8283873796463013),
 ('정상아', 0.5797291994094849),
 ('엘렉트라', 0.578797459602356),
 ('018', 0.5673673152923584),
 ('남근기', 0.5641393661499023),
 ('남자', 0.5395696759223938),
 ('여자', 0.5387108325958252),
 ('아든', 0.5367316603660583),
 ('내사시', 0.5295403003692627),
 ('경산부', 0.5278865098953247)]

In [19]:
model_all.wv.similar_by_word("여아")

[('남아', 0.8283873796463013),
 ('아든', 0.571865439414978),
 ('여자', 0.5551770925521851),
 ('018', 0.5390071868896484),
 ('남자', 0.5268032550811768),
 ('남근기', 0.5261671543121338),
 ('1317', 0.5187845230102539),
 ('여학생', 0.5119494795799255),
 ('617', 0.5077946782112122),
 ('1053', 0.5066747665405273)]

---

## Only Noun

- 이번에는 말뭉치에서 명사만 추출하여 결과를 비교해보자
- 유사한 키워드 잡기를 목적으로 할 때는 명사만 추출하는 것이 더 적절한 듯

In [20]:
%%time
def extract_noun(text):
    tokenizer = Mecab()
    text = tokenizer.nouns(text)
#     text = ' '.join(text)
    return text
    
df['noun'] = df['text'].apply(extract_noun)
df = df.dropna()

CPU times: user 1min 2s, sys: 29 s, total: 1min 31s
Wall time: 1min 29s


In [21]:
noun_corpus = np.array(df.noun)

In [22]:
%%time
model_noun = Word2Vec(noun_corpus,
                      size=200,
                      window=2,
                      workers=4,
                      min_count=5,
                      seed=1,
                      sg=1)
model_noun.save('data/tokmom_word2vec_only_noun.bin')

CPU times: user 1min 56s, sys: 502 ms, total: 1min 56s
Wall time: 33.1 s


In [23]:
model_noun.wv.similar_by_word("육아")

[('양육', 0.6242453455924988),
 ('재테크', 0.566150426864624),
 ('열혈', 0.5637132525444031),
 ('아지', 0.5588696002960205),
 ('바깥일', 0.5578585863113403),
 ('할당제', 0.552988588809967),
 ('경험자', 0.5520880818367004),
 ('독박', 0.5502913594245911),
 ('자처', 0.5501136779785156),
 ('체험담', 0.5481447577476501)]

In [24]:
model_noun.wv.similar_by_word("시댁")

[('시부모', 0.8301487565040588),
 ('처가', 0.8075655698776245),
 ('친정어머니', 0.7838934063911438),
 ('친가', 0.7829631567001343),
 ('시누이', 0.7689813375473022),
 ('시어머니', 0.7638641595840454),
 ('장모님', 0.7636337280273438),
 ('친정', 0.7625287175178528),
 ('며느리', 0.7616136074066162),
 ('식구', 0.7474988102912903)]

In [25]:
model_noun.wv.similar_by_word("놀이")

[('놀잇감', 0.6774640083312988),
 ('공놀이', 0.6645373702049255),
 ('누가누', 0.6578233242034912),
 ('줄다리기', 0.626819372177124),
 ('소꿉', 0.6250967383384705),
 ('기차놀이', 0.6238834261894226),
 ('종이접기', 0.6137405633926392),
 ('지점토', 0.6128246784210205),
 ('도미노', 0.6103137731552124),
 ('찰흙', 0.6055053472518921)]

In [26]:
model_noun.wv.similar_by_word("아이")

[('밥투정', 0.6975911855697632),
 ('지레짐작', 0.69602370262146),
 ('왜', 0.6904274225234985),
 ('갓난아이', 0.6848360896110535),
 ('갓난아기', 0.6833100318908691),
 ('말자', 0.6791629791259766),
 ('내심', 0.6775459051132202),
 ('아기', 0.6761232018470764),
 ('거려', 0.673407256603241),
 ('도통', 0.6729063391685486)]

In [27]:
model_noun.wv.similar_by_word("치아")

[('잇몸', 0.777609646320343),
 ('영구치', 0.7315409779548645),
 ('젖니', 0.7273308038711548),
 ('충치', 0.7200759649276733),
 ('어금니', 0.6887335777282715),
 ('앞니', 0.6705347299575806),
 ('유치', 0.6627084016799927),
 ('치근', 0.660078763961792),
 ('턱뼈', 0.65981525182724),
 ('사랑니', 0.6583234071731567)]

In [28]:
model_noun.wv.similar_by_word("이빨")

[('혓바닥', 0.7970603704452515),
 ('밥풀', 0.7756132483482361),
 ('거스러미', 0.7736554741859436),
 ('영인', 0.7719829082489014),
 ('덧', 0.7633000612258911),
 ('옷소매', 0.7628405094146729),
 ('속살', 0.7598446607589722),
 ('풀칠', 0.7569195628166199),
 ('혓바늘', 0.7567796111106873),
 ('비닐장갑', 0.7552425265312195)]

In [29]:
model_noun.wv.similar_by_word("감기")

[('코감기', 0.7617442011833191),
 ('목감기', 0.7377747893333435),
 ('기침감기', 0.7292241454124451),
 ('편도염', 0.7183151245117188),
 ('냉방병', 0.7179802656173706),
 ('환절기', 0.7147027254104614),
 ('병치레', 0.7000278830528259),
 ('인후염', 0.690285325050354),
 ('입병', 0.6877121925354004),
 ('잔병', 0.6861902475357056)]

In [30]:
model_noun.wv.similar_by_word("열")

[('고열', 0.6317886114120483),
 ('고열로', 0.6134417057037354),
 ('미열', 0.5859445929527283),
 ('오한', 0.5766294598579407),
 ('사병', 0.5661201477050781),
 ('불덩이', 0.5593166351318359),
 ('허열', 0.5592193007469177),
 ('해열제', 0.5514139533042908),
 ('일사병', 0.5408698320388794),
 ('외감', 0.5374668836593628)]

In [31]:
model_noun.wv.similar_by_word("아파")

[('목도', 0.8288396596908569),
 ('참기', 0.8269376754760742),
 ('어린애', 0.8175088167190552),
 ('거려', 0.8150373697280884),
 ('무의식중', 0.8126235008239746),
 ('열꽃', 0.8104689121246338),
 ('멎', 0.805420994758606),
 ('콧물감기', 0.8029375076293945),
 ('소름', 0.8007007241249084),
 ('생채기', 0.797451376914978)]

In [32]:
model_noun.wv.similar_by_word("울음")

[('큰소리', 0.6796399354934692),
 ('아달라', 0.6760480403900146),
 ('비명', 0.660555362701416),
 ('괴성', 0.6542980670928955),
 ('놀람', 0.652627170085907),
 ('옹알이', 0.6464821100234985),
 ('지르기', 0.644680380821228),
 ('십중팔구', 0.6393640041351318),
 ('세요', 0.638935923576355),
 ('신음', 0.6351783275604248)]

In [33]:
model_noun.wv.similar_by_word("남아")

[('여아', 0.8254184722900391),
 ('확연', 0.6877681612968445),
 ('생시', 0.6854578256607056),
 ('외반슬', 0.6629829406738281),
 ('임소', 0.6575838327407837),
 ('정상아', 0.6539125442504883),
 ('클리토리스', 0.6525440216064453),
 ('초혼', 0.6522287130355835),
 ('남학생', 0.6509523987770081),
 ('아포크린샘', 0.6488806009292603)]

In [34]:
model_noun.wv.similar_by_word("여아")

[('남아', 0.8254184722900391),
 ('확연', 0.6305754780769348),
 ('가슴둘레', 0.6240402460098267),
 ('생시', 0.6220796704292297),
 ('체구', 0.6026967167854309),
 ('외반슬', 0.6025059223175049),
 ('기서', 0.6004658341407776),
 ('아포크린샘', 0.5991439819335938),
 ('초혼', 0.5985881090164185),
 ('남학생', 0.5962601900100708)]

In [35]:
model_noun.wv.similar_by_word("여자")

[('남자', 0.8555517196655273),
 ('남아', 0.579403281211853),
 ('금성', 0.5788614749908447),
 ('남학생', 0.5765731930732727),
 ('조신', 0.5691870450973511),
 ('여아', 0.5655451416969299),
 ('아이든', 0.5467479228973389),
 ('얌전', 0.5369517803192139),
 ('여탕', 0.5360039472579956),
 ('야동', 0.5231473445892334)]

In [36]:
model_noun.wv.similar_by_word("남자")

[('여자', 0.8555517196655273),
 ('금성', 0.6081371307373047),
 ('조신', 0.5972491502761841),
 ('얌전', 0.5949488878250122),
 ('남학생', 0.5932546854019165),
 ('남아', 0.5725769400596619),
 ('오이디푸스', 0.5669949650764465),
 ('아이든', 0.5630729794502258),
 ('여탕', 0.5577079653739929),
 ('위엄', 0.556670069694519)]

In [37]:
model_noun.wv.similar_by_word("유모차")

[('디럭스', 0.7058776021003723),
 ('타보', 0.6657469272613525),
 ('절충', 0.658687949180603),
 ('휴대용', 0.627317488193512),
 ('페도라', 0.6180530786514282),
 ('범용', 0.6134641170501709),
 ('잉글', 0.6099952459335327),
 ('요람', 0.6077185869216919),
 ('맥클라렌', 0.6048115491867065),
 ('퀴니', 0.602877676486969)]

In [38]:
model_noun.wv.similar_by_word("카시트")

[('안전벨트', 0.6666043400764465),
 ('컨버터블', 0.6585870385169983),
 ('브라이', 0.6474909782409668),
 ('뒷좌석', 0.6399862766265869),
 ('캐리어', 0.635304868221283),
 ('부스터', 0.6323018074035645),
 ('콩코드', 0.630862832069397),
 ('페도라', 0.6296238303184509),
 ('점식', 0.6177816390991211),
 ('장착', 0.6116867661476135)]

In [39]:
model_noun.wv.most_similar(positive=["유모차", "카시트"])

[('디럭스', 0.701353907585144),
 ('페도라', 0.6979069709777832),
 ('절충', 0.6831165552139282),
 ('타보', 0.6792646050453186),
 ('휴대용', 0.679221510887146),
 ('범용', 0.6740180253982544),
 ('웨건', 0.6731740236282349),
 ('컨버터블', 0.671053946018219),
 ('브라이', 0.668348491191864),
 ('콩코드', 0.6672180891036987)]

In [40]:
model_noun.wv.most_similar(positive=["중고", "거래"])

[('반품', 0.7844113111495972),
 ('손품', 0.770588755607605),
 ('판매자', 0.7679333686828613),
 ('계약서', 0.7549364566802979),
 ('최저가', 0.7542258501052856),
 ('중개', 0.7508804798126221),
 ('영수증', 0.7379322052001953),
 ('반환', 0.7343404293060303),
 ('통관', 0.7328927516937256),
 ('매매', 0.7310436367988586)]

In [41]:
model_noun.wv.most_similar(positive=["여아", "중고"])

[('기서', 0.6915438175201416),
 ('유아복', 0.6723528504371643),
 ('충동구매', 0.671576201915741),
 ('김유진', 0.6712243556976318),
 ('중고품', 0.6674498915672302),
 ('손품', 0.6631473302841187),
 ('남아', 0.6618261933326721),
 ('내구재', 0.6601141691207886),
 ('세용', 0.6588102579116821),
 ('킬로', 0.6587986946105957)]

---

## Keywords (Noun + Verb + Adj)

- 마지막으로 말뭉치에서 명사, 동사, 형용사까지 추출한 결과를 확인

In [42]:
%%time
def extract_keywords(text):
    tokenizer = Mecab()
    pos_list = tokenizer.pos(text)
    keywords_list = list(filter(lambda w: 'NNG' in w or \
                                          'NNP' in w or \
                                          'VA' in w or \
                                          'VV' in w , pos_list))
    keywords = list(map(lambda w: w[0], keywords_list))
    return keywords

df['keywords'] = df['text'].apply(extract_keywords)
df = df.dropna()

CPU times: user 1min 5s, sys: 29.4 s, total: 1min 35s
Wall time: 1min 33s


In [43]:
keywords_corpus = np.array(df.keywords)

In [44]:
%%time
model_keywords = Word2Vec(keywords_corpus,
                          size=200,
                          window=3,
                          min_count=3,
                          workers=4,
                          seed=1,
                          sg=1)
model_keywords.save('data/tokmom_word2vec_keywords.bin')

CPU times: user 2min 42s, sys: 517 ms, total: 2min 43s
Wall time: 46.1 s


In [45]:
model_keywords.wv.similar_by_word("육아")

[('양육', 0.6082463264465332),
 ('황윤정', 0.5855332612991333),
 ('독박', 0.5838783383369446),
 ('지원군', 0.5817933082580566),
 ('열혈', 0.5815005302429199),
 ('바깥일', 0.5770851373672485),
 ('워킹맘', 0.5760881900787354),
 ('자처', 0.57264244556427),
 ('장세희', 0.567193865776062),
 ('아지', 0.5639887452125549)]

In [46]:
model_keywords.wv.similar_by_word("시댁")

[('시부모', 0.8075903058052063),
 ('처가', 0.7944771647453308),
 ('본가', 0.780774712562561),
 ('친정', 0.7743369340896606),
 ('시누이', 0.7550449371337891),
 ('장모님', 0.7505930066108704),
 ('친정어머니', 0.74419105052948),
 ('대소사', 0.7362627983093262),
 ('시어머니', 0.7321466207504272),
 ('시어머님', 0.7245830297470093)]

In [47]:
model_keywords.wv.similar_by_word("놀이")

[('놀잇감', 0.6900386214256287),
 ('공놀이', 0.6688109636306763),
 ('기차놀이', 0.650236964225769),
 ('재미있', 0.6389463543891907),
 ('소꿉', 0.6298764944076538),
 ('놀', 0.6272435188293457),
 ('샌드백', 0.6076499223709106),
 ('사방치기', 0.6030152440071106),
 ('무너뜨리기', 0.602175235748291),
 ('음률', 0.6014593839645386)]

In [48]:
model_keywords.wv.similar_by_word("아이")

[('갓난아기', 0.7090675234794617),
 ('눈치채', 0.6980432271957397),
 ('어린아이', 0.6952709555625916),
 ('운지', 0.6903558969497681),
 ('아기', 0.6865364909172058),
 ('부아', 0.6852859258651733),
 ('남부럽', 0.6839496493339539),
 ('아이라', 0.6831711530685425),
 ('유별나', 0.6789573431015015),
 ('돌쟁이', 0.6777686476707458)]

In [49]:
model_keywords.wv.similar_by_word("치아")

[('잇몸', 0.7814269065856934),
 ('충치', 0.7545057535171509),
 ('영구치', 0.7522687911987305),
 ('젖니', 0.7465424537658691),
 ('앞니', 0.70721834897995),
 ('어금니', 0.681839108467102),
 ('유치', 0.6813303828239441),
 ('턱뼈', 0.6789175271987915),
 ('법랑질', 0.6703236103057861),
 ('과잉치', 0.6679973602294922)]

In [50]:
model_keywords.wv.similar_by_word("이빨")

[('영인', 0.7976738810539246),
 ('혓바닥', 0.7932785749435425),
 ('갉', 0.7906359434127808),
 ('속니', 0.7894017696380615),
 ('근질거리', 0.7823984622955322),
 ('거스러미', 0.782036304473877),
 ('빨아먹', 0.7807981371879578),
 ('뜯어내', 0.7805253863334656),
 ('잘리', 0.7804970741271973),
 ('취기', 0.7796875834465027)]

In [51]:
model_keywords.wv.similar_by_word("감기")

[('코감기', 0.78890061378479),
 ('목감기', 0.7550700306892395),
 ('기침감기', 0.7251114845275879),
 ('냉방병', 0.7205955982208252),
 ('편도염', 0.716453492641449),
 ('열나', 0.7087898850440979),
 ('환절기', 0.7061086893081665),
 ('몸살감기', 0.6948539018630981),
 ('몸살', 0.6846282482147217),
 ('기관지염', 0.6833118796348572)]

In [52]:
model_keywords.wv.similar_by_word("열")

[('열기', 0.5785349607467651),
 ('고열로', 0.5445908904075623),
 ('오한', 0.54155433177948),
 ('외감', 0.518417477607727),
 ('미열', 0.503433108329773),
 ('고열', 0.49863749742507935),
 ('허열', 0.4985729455947876),
 ('닫', 0.49837061762809753),
 ('몸살감기', 0.49783727526664734),
 ('저체온', 0.49546051025390625)]

In [53]:
model_keywords.wv.similar_by_word("아파")

[('어린애', 0.8495914936065674),
 ('거려', 0.8362622857093811),
 ('아파하', 0.8314104080200195),
 ('참기', 0.8287290334701538),
 ('생채기', 0.8276420831680298),
 ('놀래', 0.8184535503387451),
 ('아달라', 0.8146629333496094),
 ('겁나', 0.8104078769683838),
 ('나지', 0.8078325986862183),
 ('쓰라리', 0.8066045641899109)]

In [54]:
model_keywords.wv.similar_by_word("울음")

[('우', 0.7235427498817444),
 ('자지러지', 0.6944831013679504),
 ('흐느끼', 0.6914983987808228),
 ('울', 0.6881596446037292),
 ('칭얼거리', 0.6788840889930725),
 ('아달라', 0.6761489510536194),
 ('소스라치', 0.6651831865310669),
 ('어르', 0.662794828414917),
 ('끙끙거리', 0.6609679460525513),
 ('우렁차', 0.6588179469108582)]

In [55]:
model_keywords.wv.similar_by_word("남아", topn=20)

[('여아', 0.815083920955658),
 ('클리토리스', 0.652990460395813),
 ('정상아', 0.6529625654220581),
 ('성징', 0.6491984128952026),
 ('극소', 0.6432909965515137),
 ('출생체중', 0.641374409198761),
 ('조숙', 0.6404080986976624),
 ('색맹', 0.6340155601501465),
 ('확연', 0.6320064067840576),
 ('정류', 0.6316077709197998),
 ('증후', 0.6203296184539795),
 ('볼프관', 0.6196882724761963),
 ('아도', 0.6194777488708496),
 ('위수', 0.6194471120834351),
 ('두드러지', 0.6175733208656311),
 ('표준편차', 0.6165727376937866),
 ('반항아', 0.6161479949951172),
 ('가슴둘레', 0.6157943606376648),
 ('흉위', 0.6151590347290039),
 ('생시', 0.614486038684845)]

In [56]:
model_keywords.wv.similar_by_word("여아", topn=20)

[('남아', 0.815083920955658),
 ('클리토리스', 0.5749424695968628),
 ('가슴둘레', 0.5743407607078552),
 ('여자', 0.5740675926208496),
 ('성별', 0.5651695728302002),
 ('흉위', 0.5620288252830505),
 ('생시', 0.5591805577278137),
 ('극소', 0.5552517771720886),
 ('남학생', 0.548491358757019),
 ('위수', 0.5442814230918884),
 ('성징', 0.5426771640777588),
 ('남자', 0.541545569896698),
 ('여학생', 0.5402891635894775),
 ('아도', 0.539071261882782),
 ('출생체중', 0.5377163887023926),
 ('아이든', 0.5370349287986755),
 ('정상아', 0.5312023162841797),
 ('조숙', 0.5310139656066895),
 ('반항아', 0.5290079116821289),
 ('곱슬머리', 0.5278536081314087)]

In [57]:
model_keywords.wv.similar_by_word("여자", topn=20)

[('남자', 0.8581234812736511),
 ('남아', 0.5922901630401611),
 ('조신', 0.5793790221214294),
 ('여아', 0.5740675926208496),
 ('금성', 0.5738330483436584),
 ('아이든', 0.5639742016792297),
 ('여탕', 0.5592468976974487),
 ('드세', 0.5551586747169495),
 ('성차', 0.5547588467597961),
 ('남탕', 0.5497941970825195),
 ('엘렉트라', 0.5283433198928833),
 ('화성인', 0.5268546342849731),
 ('짓궂', 0.5264767408370972),
 ('확연', 0.5199704766273499),
 ('우스갯소리', 0.5195823907852173),
 ('치이', 0.5190013647079468),
 ('들추', 0.517877459526062),
 ('객체', 0.5176793336868286),
 ('시집살이', 0.5163900256156921),
 ('에너자이저', 0.5160359740257263)]

In [58]:
model_keywords.wv.similar_by_word("남자", topn=20)

[('여자', 0.8581234812736511),
 ('조신', 0.6361842751502991),
 ('금성', 0.6076371669769287),
 ('남탕', 0.5927804708480835),
 ('남아', 0.5923527479171753),
 ('드세', 0.5884186625480652),
 ('아이든', 0.587334156036377),
 ('여탕', 0.5862170457839966),
 ('족쇄', 0.5841605067253113),
 ('얌전', 0.5800723433494568),
 ('치이', 0.5791993141174316),
 ('객체', 0.5764616131782532),
 ('들추', 0.5746737718582153),
 ('성차', 0.5679875016212463),
 ('짓궂', 0.563059389591217),
 ('우스갯소리', 0.561324954032898),
 ('속사정', 0.5608287453651428),
 ('엘렉트라', 0.5591105222702026),
 ('화성인', 0.5560164451599121),
 ('철없', 0.5548366904258728)]

In [59]:
model_keywords.wv.similar_by_word("유모차")

[('디럭스', 0.7637549638748169),
 ('절충', 0.7002888917922974),
 ('휴대용', 0.6874918937683105),
 ('잉글', 0.665499210357666),
 ('타보', 0.6651328206062317),
 ('레시', 0.6447641849517822),
 ('핸들링', 0.6435951590538025),
 ('포투', 0.636689305305481),
 ('웨건', 0.6357094049453735),
 ('맥클라렌', 0.6271377801895142)]

In [60]:
model_keywords.wv.most_similar(positive=["중고", "유모차"])

[('휴대용', 0.7081874012947083),
 ('디럭스', 0.6892330646514893),
 ('렌터카', 0.687420666217804),
 ('범용', 0.6817907094955444),
 ('중저가', 0.6776123046875),
 ('최저가', 0.6769124269485474),
 ('내구재', 0.6753764152526855),
 ('초경량', 0.6748402118682861),
 ('웨건', 0.6729285717010498),
 ('맥클라렌', 0.6727186441421509)]

In [61]:
model_keywords.wv.most_similar(positive=["좋", "유모차"])

[('초경량', 0.6407731771469116),
 ('차광막', 0.6365723013877869),
 ('기동력', 0.6358193159103394),
 ('체크포인트', 0.634564995765686),
 ('연습기', 0.6332307457923889),
 ('무거우', 0.632172703742981),
 ('행선지', 0.6308422088623047),
 ('플루토', 0.6299110651016235),
 ('소서', 0.6295733451843262),
 ('커텐', 0.627544641494751)]

In [62]:
model_keywords.wv.most_similar(positive=["안", "좋", "유모차"])

[('차광막', 0.7198842763900757),
 ('무거우', 0.7128297090530396),
 ('이동도', 0.6930224895477295),
 ('촉박', 0.6849249601364136),
 ('침대위', 0.683809757232666),
 ('머미', 0.6802859306335449),
 ('비켜나', 0.6796587109565735),
 ('허리끈', 0.6794095635414124),
 ('남아나', 0.678951621055603),
 ('베게', 0.6787242293357849)]

In [63]:
model_keywords.wv.most_similar(positive=["안", "좋"])

[('내어보', 0.655441164970398),
 ('차면', 0.6488319635391235),
 ('촉박', 0.6480374932289124),
 ('황길', 0.6475943326950073),
 ('산보', 0.6470712423324585),
 ('졸립', 0.6424490809440613),
 ('목마르', 0.6404247283935547),
 ('덮어두', 0.6400163173675537),
 ('올게', 0.6396368741989136),
 ('쪼이', 0.6382529139518738)]

In [64]:
model_keywords.wv.most_similar(positive=["가슴골"])

[('삼각근', 0.9021599292755127),
 ('비벼대', 0.901026725769043),
 ('온습포', 0.8989864587783813),
 ('떠받치', 0.8962262272834778),
 ('반고리관', 0.8962064385414124),
 ('땡기', 0.8945552110671997),
 ('요탕', 0.8937971591949463),
 ('재본', 0.8918069005012512),
 ('쳐지', 0.8916620016098022),
 ('태충', 0.8916598558425903)]

---

### Volume of Corpus 

In [65]:
df[['text']].to_csv('data/text_for_word2vec.csv', index=False, encoding='utf-8')

In [66]:
pd.read_csv('data/text_for_word2vec.csv')

,text
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...
...,...
24724,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...
24725,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...
24726,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...
24727,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...


---

## Doc2Vec

In [67]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [68]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(df['noun'])]

In [69]:
%%time
doc2vec_model = Doc2Vec(documents, 
                        vector_size=200, 
                        window=2, 
                        min_count=5, 
                        workers=4)


doc2vec_model.save('data/tokmom_doc2vec_only_noun.bin')

CPU times: user 1min 9s, sys: 1.22 s, total: 1min 10s
Wall time: 28.9 s


In [70]:
len(doc2vec_model.docvecs)

24729

In [71]:
df['text'][0]

'6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 사촌 형 누나와 함께 자랐어요 금요일 저녁에 데려와 주말에는 엄마 아빠와 생활하고 있구요 7세가 되는 내년이라도 아이를 데려오자는 아이 아빠의 의견과 유치원과 태권도 등 패턴이 적응되었고 형 누나들과 더 자랄 수 있게 초등학교 입학시기에 데리고 가는게 낫지 않겠냐는 외할머니 의견이 다릅니다 6세가 된 아이의 교육과 생활습관 형성이 이제는 할머니댁에서는 어려울 것 같다는 생각이 들고 하루하루 생각이 크고 자라는 아이를 주말에만 보는 저 또한 놀랄 때가 많거든요 하루라도 빨리 데려와야 할까요 초등학교 입학시기에 데려오는 편이 아이에게 좋을까요 매우 어려운 질문입니다 먼저 아이가 현재 잘 적응하고 있고 주변 양육환경 역시 아이가 성장하기에 별다른 문제가 없다 하니 고민이 더욱 크시리라 생각됩니다 우선 되어야하는 것은 아이의 의견입니다 6살이면 이러한 문제에 대하여 스스로 생각하고 의견을 전달할 수 있으므로 먼저 아이의 의견을 들어 주시기 바랍니다 아이는 부모님과 함께 사는 것이 좋습니다 그러나 부모님께서 두분 모두 직장에 나가시니 유치원 하원 후 아이를 돌보아 주실 분이 현재 함께 생활하고 계시는 할머님이시면 재고의 여지 없이 데려 오시는 것을 권유 드립니다 그러나 하원 후 할머님의 돌봄이 여의치 못할 경우 돌보아 줄 대상이 당분간 할머님댁에서 시간제로나마 아이를 돌보아 주어 아이와의 상호작용을 충분히 가진 후에 아이가 집으로 왔을 때 유치원 하원 후에 계속 돌보아 줄 수 있으면 좋습니다 이와 같이 먼저 아이가 집으로 온 후 부모님께서 퇴근하시기 전 까지 돌보아 줄 사람을 알아보시고 데려 오시는 것이 좋다는 생각이 듭니다'

In [72]:
print("Input Document", df['text'][100], sep="\n")

similar_docs = doc2vec_model.docvecs.most_similar(100)

print("Most similar Documents")
for idx, similarity in similar_docs[:3]:
    print("idx:", idx, "similarity:", np.round(similarity, 3))
    print(df['text'][idx])

Input Document
젖병을 떼야 하는 이유로는 첫째 우식증을 예방하고 둘째 젖병 무는 어린 아기의 습관을 서서히 고쳐 나가기 위함 입니다 지금은 단지 습관에 의해 젖병을 물고 있을 뿐이므로 음식은 숟가락으로 먹이고 우유는 컵으로 마시게 하면서 젖병을 뗄 수 있도록 해야 합니다 잠잘 때 젖병을 물고 자는 것은 무의식적인 행동입니다 그러나 너무 친숙해진 습관을 갑자기 금지시킨다면 아기 심리에 큰 충격을 줄 수도 있으므로 서서히 젖병 속의 우유를 희석시키는 방법을 사용합니다 예를 들어 평상시에 우유를 200 mL 먹였다면 우유 180 mL에 물 20 mL를 섞어서 젖병에 넣어줍니다 점차 물의 농도를 높여가다가 아기가 이를 느끼고 부정적인 반응을 보일 때는 잠시 정지하였다가 다시 희석의 강도를 높여봅니다 그러다 보면 아기는 점차 희석된 우유에 흥미를 잃어 스스로 젖병을 내던지게 될 것입니다 아기에 따라 스스로 정서적 안정을 얻는 습관을 하나씩 가지고 있습니다 아기 때의 이불을 늘 잡고 있다거나 엄마의 머리카락을 잡아야 잠을 잔다거나 하는 습관은 고치기가 쉽지 않은 일입니다 젖병도 그런 습관의 일종일 수 있습니다 젖병을 물고 있는 것이 정서적인 이유라면 젖병 떼기를 강요하지 않습니다 하루 2 3번은 젖병을 물고 있게 하다가 점차 아기가 성장하면서 젖병 무는 것이 자신에게는 어울리지 않는다는 것을 엄마가 끊임없이 설득하고 아이 스스로도 그렇게 느껴 자연스레 젖병을 멀리하게 합니다 가능한 컵이나 빨대컵을 활용하고 노리개 젖꼭지 등을 대체하여 사용해 볼 수도 있습니다
Most similar Documents
idx: 15095 similarity: 0.666
아기가 젖병만 보면 입으로 가져가는 이유는 왜일까 배가 고프다는 의사 표현일까 아니면 말랑말랑 젤리 같은 젖꼭지의 느낌이 좋아서일까 혹시 어딘가 아프거나 불만이 있다는 무언의 표현은 아닐까 배가 고프면 엄마를 찾거나 우유를 찾는 것은 당연한 일이겠지만 아기들은 배가 고프지 않아도 젖병을 물고 싶어한다 별다른 이유

In [73]:
df['tokenized_doc'] = df['noun'].apply(lambda x: doc2vec_model.infer_vector(x).astype(float))

In [74]:
df

,text,tokenized,noun,keywords,tokenized_doc
0,6살 남아 입니다 회사 복직 후 3살부터 주중에는 외할머니댁에서 2살 4살 터울의 ...,"[6, 살, 남, 아, 입, 니, 다, 회사, 복직, 후, 3, 살, 부터, 주, ...","[살, 회사, 복직, 후, 살, 주, 중, 외할머니, 댁, 살, 살, 터울, 사촌,...","[남, 입, 회사, 복직, 후, 주, 외할머니, 댁, 터울, 사촌, 형, 누나, 금...","[-0.020082682371139526, 0.4280707538127899, -0..."
1,요즘 들어 아빠가 아이에게 다가가려고 하면 엄마한테 갈 거야 하면서 엄마를 유독 찾...,"[요즘, 들, 어, 아빠, 가, 아이, 에게, 다가가, 려고, 하, 면, 엄마, 한...","[아빠, 아이, 엄마, 거, 엄마, 전, 정도, 엄마, 아빠, 거부, 엄마, 눈, ...","[들, 아빠, 아이, 다가가, 하, 엄마, 하, 엄마, 찾, 전, 정도, 엄마, 매...","[0.1081785038113594, 0.4164026379585266, -0.15..."
2,둘째가 태어나면 첫째가 힘들 거라는 건 예상하고 있었지만 툭하면 삐치고 울고 좀처럼...,"[둘째, 가, 태어나, 면, 첫째, 가, 힘들, 거, 라는, 건, 예상, 하, 고,...","[둘째, 첫째, 거, 건, 예상, 때, 엄마, 껌, 딱지, 첫째, 외출, 때, 제외...","[태어나, 예상, 삐치, 울, 서럽, 웁, 때, 엄마, 껌, 딱지, 외출, 때, 제...","[0.11574923247098923, 0.1414957493543625, -0.2..."
3,20개월 남아가 있고 임신 8개월차로 두 아기의 엄마입니다 첫 아이는 잠자리 독립을...,"[20, 개월, 남아, 가, 있, 고, 임신, 8, 개월, 차, 로, 두, 아기, ...","[개월, 남아, 임신, 개월, 차, 아기, 엄마, 아이, 잠자리, 독립, 생후, 일...","[남아, 있, 임신, 차, 아기, 엄마, 아이, 잠자리, 독립, 생후, 전, 재우,...","[0.2988924980163574, 0.33812403678894043, -0.5..."
4,14개월 남자 아기인데 심하게 깨무는 버릇이 있어요 전에는 졸릴 때 주로 물곤 했는...,"[14, 개월, 남자, 아기, 인데, 심하, 게, 깨무, 는, 버릇, 이, 있, 어...","[개월, 남자, 아기, 버릇, 전, 때, 이제, 자기, 지대, 때, 떼, 상대방, ...","[남자, 아기, 심하, 깨무, 버릇, 있, 전, 때, 물, 이제, 지대, 때, 떼,...","[0.01197979785501957, 0.26562419533729553, -0...."
...,...,...,...,...,...
24724,가임기 연력 여성의 흡연률 증가와 그 폐해 현대 여성은 다양한 사회활동에 참여하고 ...,"[가, 임기, 연력, 여성, 의, 흡연, 률, 증가, 와, 그, 폐해, 현대, 여성...","[임기, 연력, 여성, 흡연, 증가, 폐해, 현대, 여성, 사회, 활동, 참여, 경...","[임기, 연력, 여성, 흡연, 증가, 폐해, 현대, 여성, 사회, 활동, 참여, 경...","[0.23498819768428802, -0.5549213290214539, -0...."
24725,아토피에 유전적인 요인이 작용한다는 사실 알고 계시나요 유전적인 요인 뿐 아니라 아...,"[아토피, 에, 유전, 적, 인, 요인, 이, 작용, 한다는, 사실, 알, 고, 계...","[아토피, 유전, 요인, 작용, 사실, 유전, 요인, 뿐, 아토피, 유발, 수, 하...","[아토피, 유전, 요인, 작용, 사실, 알, 유전, 요인, 아토피, 유발, 있, 중...","[-0.09013593196868896, 0.16884320974349976, -0..."
24726,1st 시기에 따라 필요한 영양소를 충분히 뱃속의 아이는 엄마가 먹는 것을 함께 먹...,"[1, st, 시기, 에, 따라, 필요, 한, 영양소, 를, 충분히, 뱃속, 의, ...","[시기, 필요, 영양소, 뱃속, 아이, 엄마, 것, 균형, 영양, 음식, 섭취, 태...","[시기, 필요, 영양소, 뱃속, 아이, 엄마, 먹, 먹, 균형, 잡히, 영양, 있,...","[0.3577316701412201, 0.5504094958305359, -0.28..."
24727,분만 여성만이 누릴 수 있는 특권 대부분의 여성들은 임신과 분만이라는 과정을 경험하...,"[분만, 여성, 만, 이, 누릴, 수, 있, 는, 특권, 대부분, 의, 여성, 들,...","[분만, 여성, 수, 특권, 대부분, 여성, 임신, 분만, 과정, 경험, 삶, 의미...","[분만, 여성, 있, 특권, 대부분, 여성, 임신, 분만, 과정, 경험, 삶, 의미...","[0.21933366358280182, 0.12538917362689972, -0...."
